In [1]:
import torch
from torch.backends import cudnn

from my_utils import data, models
from my_utils.utils import save_model, check_accuracy, train
from my_utils.vis import *

In [12]:
print(torch.cuda.get_device_properties(0))
print(cudnn.is_available())
a = torch.tensor(1.)
a.cuda(device='cuda:1')

_CudaDeviceProperties(name='Tesla V100-SXM2-32GB', major=7, minor=0, total_memory=32510MB, multi_processor_count=80)
True


tensor(1., device='cuda:1')

In [ ]:
import os

In [41]:
from scipy.io import loadmat
import h5py

ROOT = '/home/wangyh/02-Datasets/Z-NLOS'
PATH = os.path.join(ROOT, os.listdir(ROOT)[0])
with h5py.File(PATH, 'r') as f:
    print(f.keys())
    print(f['data'])
# data = h5py.File('matlabdata.mat')['matlabdata'].value
# data

<KeysViewHDF5 ['cameraGridNormals', 'cameraGridPoints', 'cameraGridPositions', 'cameraGridSize', 'cameraPosition', 'data', 'deltaT', 'hiddenVolumePosition', 'hiddenVolumeRotation', 'hiddenVolumeSize', 'isConfocal', 'laserGridNormals', 'laserGridPoints', 'laserGridPositions', 'laserGridSize', 'laserPosition', 't', 't0']>
<HDF5 dataset "data": shape (1024, 6, 256, 256), type "<f4">
